In [23]:
from src.rel_prop.min_max_rel_model import MinMaxModel, Nested_Regressor
from src.models.Binary_Mnist_Model import Montavon_Classifier
import numpy as np
from src.rel_prop.minmax_utils import get_higher_relevances
import matplotlib.pyplot as plt

In [24]:
mc = Montavon_Classifier(class_nb=8, load_model=False)
mc.set_data(test_size=0.2)
mc.set_model()
mc.model.summary()
mc.fit_model(epochs=100, batch_size=32)


Train images: 21840, train labels: 21840
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 400)               313600    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               40000     
_________________________________________________________________
dense_2 (Dense)              (None, 400)               40000     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 400       
Total params: 394,000
Trainable params: 353,600
Non-trainable params: 40,400
_________________________________________________________________
Model: "sequential"
__________________________________________________

KeyboardInterrupt: 

In [ ]:
# use to recalc relevances for the training of the min max model. Therefore set recalc_rel to True


In [ ]:
minmax = MinMaxModel(classifier=mc, use_higher_rel=True)



In [ ]:
nr_train_images = minmax.nr_train_images
true_relevances = minmax.true_relevances
higher_relevances = minmax.higher_relevances

minmax.train_min_max(pretrained=False)



In [ ]:
idx = np.random.randint(0,100)
print(idx)
final_relevance, zplus_relevance = minmax.min_max_rel_prop(idx)
a = 1

In [ ]:
import numpy as np
from src.plotting.plot_funcs import plotly_mnist_image
final_relevance = final_relevance[0]
plotly_mnist_image(final_relevance[0])